In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
trips = pd.read_csv('data/trips.csv')

In [4]:
utility = pd.read_csv('data/utilityvars.csv')

In [5]:
utility = utility.drop(columns=['activityid', 'tourmode'])
utility

,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,tourpurpose,firststop,laststop,zerototalstops,targettripmode
0,4,True,2,2,4,9.633200,11.594058,4.0,67.659134,5.053318,5.053318,5.053318,0.0,0.0,0.0,10.677524,-999.0,-999.0,0.0,0.0,46.447229,46.447229,46.447229,0,True,False,False,1
1,6,False,2,2,1,8.304700,70.117912,12.0,32.273338,31.226185,31.226185,31.226185,0.0,0.0,0.0,113.844883,-999.0,-999.0,0.0,0.0,495.225231,495.225231,495.225231,0,False,False,False,1
2,6,False,2,6,3,10.189500,47.538746,3.0,13.567157,30.145569,30.145569,30.145569,0.0,0.0,0.0,106.238420,-999.0,-999.0,0.0,0.0,462.137117,462.137117,462.137117,0,False,False,True,1
3,4,False,2,1,2,13.393200,21.983759,4.0,51.544109,24.318232,24.318232,24.318232,0.0,0.0,0.0,78.687684,-999.0,-999.0,0.0,0.0,342.291418,342.291418,342.291418,0,False,False,False,1
4,6,False,2,3,2,13.274000,6.568953,3.0,8.480003,25.267041,25.267041,25.267041,0.0,0.0,0.0,71.307144,-999.0,-999.0,0.0,0.0,310.186071,310.186071,310.186071,0,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14047024,2,True,2,4,2,35.460899,286.328949,14.0,12.385718,56.546418,56.546418,56.546418,0.0,0.0,0.0,193.717741,-999.0,-999.0,0.0,0.0,842.672156,842.672156,842.672156,5,False,False,False,5
14047025,7,False,2,4,3,17.207199,19.868286,13.0,1.423494,1.403090,1.403090,1.403090,0.0,0.0,0.0,3.069109,-999.0,-999.0,0.0,0.0,13.350623,13.350623,13.350623,5,False,False,False,3
14047026,7,False,2,4,3,16.574200,1.423494,13.0,88.151642,12.287907,12.287907,12.287907,0.0,0.0,0.0,31.580930,-999.0,-999.0,0.0,0.0,137.377041,137.377041,137.377041,5,False,False,False,3
14047027,2,True,2,4,2,22.916100,12.385718,14.0,8.472905,1.173692,1.173692,1.173692,0.0,0.0,0.0,1.252347,-999.0,-999.0,0.0,0.0,5.447711,5.447711,5.447711,5,False,False,False,3


In [7]:
# sample the whole dataset

mode1 = utility[utility['targettripmode']==1].sample(n=1000, random_state=1)
mode2 = utility[utility['targettripmode']==2].sample(n=1000, random_state=1)
mode3 = utility[utility['targettripmode']==3].sample(n=1000, random_state=1)
mode4 = utility[utility['targettripmode']==4].sample(n=1000, random_state=1)
mode5 = utility[utility['targettripmode']==5].sample(n=1000, random_state=1)
mode6 = utility[utility['targettripmode']==6].sample(n=1000, random_state=1)
mode7 = utility[utility['targettripmode']==7].sample(n=1000, random_state=1)
mode8 = utility[utility['targettripmode']==8].sample(n=1000, random_state=1)
mode9 = utility[utility['targettripmode']==9].sample(n=1000, random_state=1)
mode10 = utility[utility['targettripmode']==10].sample(n=1000, random_state=1)
mode11 = utility[utility['targettripmode']==11].sample(n=1000, random_state=1)
mode12 = utility[utility['targettripmode']==12].sample(n=1000, random_state=1)
combined_subsamples = pd.concat([mode1,mode2,mode3,mode4,mode5,mode6,mode7,mode8,mode9,mode10,mode11,mode12], axis=0)

In [8]:
# One-hot encode the categorical variables (the variables only include 0 and -1 don't need to be encoded)
new_utilityvars = pd.get_dummies(combined_subsamples, columns=["tourpurpose"], prefix=["tourpurpose"])
new_utilityvars

,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,firststop,laststop,zerototalstops,targettripmode,tourpurpose_0,tourpurpose_1,tourpurpose_2,tourpurpose_3,tourpurpose_4,tourpurpose_5,tourpurpose_6,tourpurpose_7
389395,6,True,2,1,3,4.7632,27.730175,23.0,24.913393,5.347993,5.347993,5.347993,0.0,0.0,0.0,8.340726,-999.0,-999.0,0.0,0.0,36.282156,36.282156,36.282156,True,False,False,1,0,0,0,0,1,0,0,0
9670948,7,True,1,4,4,9.2246,15.671348,6.0,31.653795,2.035063,2.035063,2.035063,0.0,0.0,0.0,3.962483,-999.0,-999.0,0.0,0.0,17.236800,17.236800,17.236800,False,False,False,1,0,0,0,0,0,1,0,0
12239774,6,True,2,4,2,17.5495,41.898407,13.0,2.250410,14.014216,14.014216,14.014216,0.0,0.0,0.0,45.059798,-999.0,-999.0,0.0,0.0,196.010116,196.010116,196.010116,False,False,True,1,0,0,0,0,1,0,0,0
8333334,7,False,2,2,3,6.9557,33.792797,1.0,3.054741,14.571569,14.571569,14.571569,0.0,0.0,0.0,48.703855,-999.0,-999.0,0.0,0.0,211.861765,211.861765,211.861765,False,False,False,1,1,0,0,0,0,0,0,0
5401543,7,True,1,2,1,11.6963,5.626131,6.0,6.404369,6.707609,6.707609,6.707609,0.0,0.0,0.0,18.757141,-999.0,-999.0,0.0,0.0,81.593560,81.593560,81.593560,False,False,False,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237872,1,True,2,4,3,4.7861,13.212724,23.0,14.727474,16.225039,15.977013,16.225039,0.0,0.0,0.0,41.776687,-999.0,-999.0,0.0,0.0,181.728585,180.567631,181.728585,False,False,True,12,0,0,1,0,0,0,0,0
540386,2,True,2,6,4,5.3317,5.763243,8.0,25.365267,7.510640,7.510640,7.510640,0.0,0.0,0.0,18.147925,-999.0,-999.0,0.0,0.0,78.943471,78.943471,78.943471,False,False,True,12,0,0,1,0,0,0,0,0
1870185,2,False,1,4,5,11.0664,13.881120,6.0,68.919067,4.094756,4.094756,4.094756,0.0,0.0,0.0,10.233641,-999.0,-999.0,0.0,0.0,44.516339,44.516339,44.516339,False,False,True,12,0,0,1,0,0,0,0,0
7502625,1,True,1,7,4,8.3753,58.655903,3.0,12.468121,5.849214,5.849214,5.849214,0.0,0.0,0.0,15.680112,-999.0,-999.0,0.0,0.0,68.208487,68.208487,68.208487,False,False,True,12,0,0,1,0,0,0,0,0


In [14]:
middle_income_grp = new_utilityvars[new_utilityvars['income'] == 3]

In [20]:
low_income_grp = new_utilityvars[new_utilityvars['income'].isin([1,2])]
high_income_grp = new_utilityvars[new_utilityvars['income'].isin([4,5])] 

In [67]:
low_income_grp.shape

(4607, 35)

In [68]:
middle_income_grp.shape

(2482, 35)

In [66]:
high_income_grp.shape

(4911, 35)

In [9]:
# Build XGBoost tree model
new_utilityvars = pd.merge(new_utilityvars.drop(['targettripmode'], axis=1), new_utilityvars[['targettripmode']], left_index=True, right_index=True, how="outer")
new_utilityvars

,age,gender,autosuf,numhouseholdpersons,income,oduden,oempden,ototint,dempden,sovdrivetime,hovdrivetime,tolldrivetime,tollcostsov,tollcosthov2,tollcosthov3,walkttime,walktotransitutility,drivetotransitutility,parkingcost,parkingwalktime,sovcost,hovcost,tollcost,firststop,laststop,zerototalstops,tourpurpose_0,tourpurpose_1,tourpurpose_2,tourpurpose_3,tourpurpose_4,tourpurpose_5,tourpurpose_6,tourpurpose_7,targettripmode
389395,6,True,2,1,3,4.7632,27.730175,23.0,24.913393,5.347993,5.347993,5.347993,0.0,0.0,0.0,8.340726,-999.0,-999.0,0.0,0.0,36.282156,36.282156,36.282156,True,False,False,0,0,0,0,1,0,0,0,1
9670948,7,True,1,4,4,9.2246,15.671348,6.0,31.653795,2.035063,2.035063,2.035063,0.0,0.0,0.0,3.962483,-999.0,-999.0,0.0,0.0,17.236800,17.236800,17.236800,False,False,False,0,0,0,0,0,1,0,0,1
12239774,6,True,2,4,2,17.5495,41.898407,13.0,2.250410,14.014216,14.014216,14.014216,0.0,0.0,0.0,45.059798,-999.0,-999.0,0.0,0.0,196.010116,196.010116,196.010116,False,False,True,0,0,0,0,1,0,0,0,1
8333334,7,False,2,2,3,6.9557,33.792797,1.0,3.054741,14.571569,14.571569,14.571569,0.0,0.0,0.0,48.703855,-999.0,-999.0,0.0,0.0,211.861765,211.861765,211.861765,False,False,False,1,0,0,0,0,0,0,0,1
5401543,7,True,1,2,1,11.6963,5.626131,6.0,6.404369,6.707609,6.707609,6.707609,0.0,0.0,0.0,18.757141,-999.0,-999.0,0.0,0.0,81.593560,81.593560,81.593560,False,False,False,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237872,1,True,2,4,3,4.7861,13.212724,23.0,14.727474,16.225039,15.977013,16.225039,0.0,0.0,0.0,41.776687,-999.0,-999.0,0.0,0.0,181.728585,180.567631,181.728585,False,False,True,0,0,1,0,0,0,0,0,12
540386,2,True,2,6,4,5.3317,5.763243,8.0,25.365267,7.510640,7.510640,7.510640,0.0,0.0,0.0,18.147925,-999.0,-999.0,0.0,0.0,78.943471,78.943471,78.943471,False,False,True,0,0,1,0,0,0,0,0,12
1870185,2,False,1,4,5,11.0664,13.881120,6.0,68.919067,4.094756,4.094756,4.094756,0.0,0.0,0.0,10.233641,-999.0,-999.0,0.0,0.0,44.516339,44.516339,44.516339,False,False,True,0,0,1,0,0,0,0,0,12
7502625,1,True,1,7,4,8.3753,58.655903,3.0,12.468121,5.849214,5.849214,5.849214,0.0,0.0,0.0,15.680112,-999.0,-999.0,0.0,0.0,68.208487,68.208487,68.208487,False,False,True,0,0,1,0,0,0,0,0,12


In [10]:
df_array = new_utilityvars.values
X = df_array[:,0:34]
Y = df_array[:,34]

In [11]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [23]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 0)

/home/likuwaha/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:17:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=64, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
y_pred = [round(value) for value in model.predict(X_test)]
y_test = [round(value) for value in y_test]

In [25]:
# multilabel confusion matrix
metrics.multilabel_confusion_matrix(y_test, y_pred)

array([[[3429,  201],
        [ 184,  146]],

       [[3506,  140],
        [ 104,  210]],

       [[3447,  173],
        [ 263,   77]],

       [[3458,  165],
        [ 216,  121]],

       [[3426,  194],
        [ 195,  145]],

       [[3469,  153],
        [ 148,  190]],

       [[3421,  207],
        [ 168,  164]],

       [[3513,  132],
        [ 149,  166]],

       [[3462,  157],
        [ 242,   99]],

       [[3411,  225],
        [ 142,  182]],

       [[3466,  176],
        [ 196,  122]],

       [[3529,  100],
        [  16,  315]]])

In [26]:
# 13*13 confusion matrix
metrics.confusion_matrix(y_test, y_pred)

array([[146,   2,  36,   2,  29,   1,  26,  24,  14,  38,  12,   0],
       [  3, 210,   0,  67,   1,  29,   0,   0,   1,   3,   0,   0],
       [ 55,   1,  77,   3,  84,   3,  36,  10,  21,   9,  23,  18],
       [  6,  94,   4, 121,   3,  96,   3,   0,   2,   6,   1,   1],
       [ 18,   1,  59,   3, 145,   8,  38,   7,  16,   8,  21,  16],
       [  3,  35,   7,  78,   9, 190,   2,   0,   1,   4,   6,   3],
       [ 20,   1,  26,   2,  23,   3, 164,  38,  27,   1,   8,  19],
       [ 27,   1,  10,   1,  10,   1,  45, 166,  20,  16,   7,  11],
       [ 34,   1,  15,   0,  12,   1,  37,  21,  99,  68,  38,  15],
       [ 20,   3,   3,   3,   2,   6,   9,  18,  22, 182,  56,   0],
       [ 15,   1,  11,   6,  21,   5,   9,   7,  32,  72, 122,  17],
       [  0,   0,   2,   0,   0,   0,   2,   7,   1,   0,   4, 315]])

In [30]:
 metrics.confusion_matrix(y_test, y_pred)/ metrics.confusion_matrix(y_test, y_pred).astype(np.float).sum(axis=1)

array([[0.44242424, 0.00636943, 0.10588235, 0.00593472, 0.08529412,
        0.00295858, 0.07831325, 0.07619048, 0.04105572, 0.11728395,
        0.03773585, 0.        ],
       [0.00909091, 0.66878981, 0.        , 0.19881306, 0.00294118,
        0.08579882, 0.        , 0.        , 0.00293255, 0.00925926,
        0.        , 0.        ],
       [0.16666667, 0.00318471, 0.22647059, 0.00890208, 0.24705882,
        0.00887574, 0.10843373, 0.03174603, 0.06158358, 0.02777778,
        0.07232704, 0.05438066],
       [0.01818182, 0.29936306, 0.01176471, 0.35905045, 0.00882353,
        0.28402367, 0.00903614, 0.        , 0.0058651 , 0.01851852,
        0.00314465, 0.00302115],
       [0.05454545, 0.00318471, 0.17352941, 0.00890208, 0.42647059,
        0.02366864, 0.11445783, 0.02222222, 0.04692082, 0.02469136,
        0.06603774, 0.04833837],
       [0.00909091, 0.11146497, 0.02058824, 0.23145401, 0.02647059,
        0.56213018, 0.0060241 , 0.        , 0.00293255, 0.01234568,
        0.01886792,

In [36]:
normalizeddf = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred)/ metrics.confusion_matrix(y_test, y_pred).astype(np.float).sum(axis=1))

In [38]:
normalizeddf.columns = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf.index = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']

In [39]:
normalizeddf

,pred Drive Alone Free,pred Drive Alone Pay,pred HOV2 Free,pred HOV2 Pay,pred HOV3 Free,pred HOV3 Pay,pred Walk,pred BIKE,pred Walk to Transit,pred Park and Ride,pred Kiss and Ride,pred School Bus
pred Drive Alone Free,0.442424,0.006369,0.105882,0.005935,0.085294,0.002959,0.078313,0.076190,0.041056,0.117284,0.037736,0.000000
pred Drive Alone Pay,0.009091,0.668790,0.000000,0.198813,0.002941,0.085799,0.000000,0.000000,0.002933,0.009259,0.000000,0.000000
pred HOV2 Free,0.166667,0.003185,0.226471,0.008902,0.247059,0.008876,0.108434,0.031746,0.061584,0.027778,0.072327,0.054381
pred HOV2 Pay,0.018182,0.299363,0.011765,0.359050,0.008824,0.284024,0.009036,0.000000,0.005865,0.018519,0.003145,0.003021
pred HOV3 Free,0.054545,0.003185,0.173529,0.008902,0.426471,0.023669,0.114458,0.022222,0.046921,0.024691,0.066038,0.048338
pred HOV3 Pay,0.009091,0.111465,0.020588,0.231454,0.026471,0.562130,0.006024,0.000000,0.002933,0.012346,0.018868,0.009063
pred Walk,0.060606,0.003185,0.076471,0.005935,0.067647,0.008876,0.493976,0.120635,0.079179,0.003086,0.025157,0.057402
pred BIKE,0.081818,0.003185,0.029412,0.002967,0.029412,0.002959,0.135542,0.526984,0.058651,0.049383,0.022013,0.033233
pred Walk to Transit,0.103030,0.003185,0.044118,0.000000,0.035294,0.002959,0.111446,0.066667,0.290323,0.209877,0.119497,0.045317
pred Park and Ride,0.060606,0.009554,0.008824,0.008902,0.005882,0.017751,0.027108,0.057143,0.064516,0.561728,0.176101,0.000000


In [41]:
# for the middle income group
df_array = middle_income_grp.values
X = df_array[:,0:34]
Y = df_array[:,34]

In [42]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [43]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 0)

/home/likuwaha/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:25:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=64, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [44]:
y_pred = [round(value) for value in model.predict(X_test)]
y_test = [round(value) for value in y_test]

In [46]:
normalizeddf = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred)/ metrics.confusion_matrix(y_test, y_pred).astype(np.float).sum(axis=1))
normalizeddf.columns = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf.index = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf

,pred Drive Alone Free,pred Drive Alone Pay,pred HOV2 Free,pred HOV2 Pay,pred HOV3 Free,pred HOV3 Pay,pred Walk,pred BIKE,pred Walk to Transit,pred Park and Ride,pred Kiss and Ride,pred School Bus
pred Drive Alone Free,0.397260,0.025316,0.144578,0.000000,0.169811,0.000000,0.033898,0.056604,0.074074,0.123288,0.038462,0.000000
pred Drive Alone Pay,0.013699,0.632911,0.012048,0.266667,0.018868,0.098765,0.000000,0.000000,0.018519,0.013699,0.000000,0.000000
pred HOV2 Free,0.123288,0.000000,0.216867,0.000000,0.377358,0.000000,0.220339,0.094340,0.037037,0.068493,0.076923,0.067568
pred HOV2 Pay,0.013699,0.189873,0.036145,0.366667,0.000000,0.222222,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000
pred HOV3 Free,0.000000,0.000000,0.132530,0.000000,0.396226,0.012346,0.067797,0.018868,0.037037,0.013699,0.076923,0.081081
pred HOV3 Pay,0.000000,0.151899,0.000000,0.300000,0.113208,0.506173,0.000000,0.000000,0.000000,0.013699,0.038462,0.000000
pred Walk,0.054795,0.000000,0.072289,0.000000,0.037736,0.012346,0.440678,0.150943,0.074074,0.000000,0.038462,0.067568
pred BIKE,0.123288,0.012658,0.012048,0.000000,0.018868,0.000000,0.084746,0.415094,0.111111,0.054795,0.038462,0.013514
pred Walk to Transit,0.095890,0.000000,0.024096,0.016667,0.094340,0.000000,0.067797,0.094340,0.240741,0.150685,0.064103,0.013514
pred Park and Ride,0.054795,0.000000,0.012048,0.016667,0.000000,0.000000,0.016949,0.075472,0.148148,0.547945,0.179487,0.000000


In [47]:
# for the low income group
df_array = low_income_grp.values
X = df_array[:,0:34]
Y = df_array[:,34]

In [48]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [49]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 0)

/home/likuwaha/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:42:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=64, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [50]:
y_pred = [round(value) for value in model.predict(X_test)]
y_test = [round(value) for value in y_test]

In [51]:
normalizeddf = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred)/ metrics.confusion_matrix(y_test, y_pred).astype(np.float).sum(axis=1))
normalizeddf.columns = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf.index = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf

,pred Drive Alone Free,pred Drive Alone Pay,pred HOV2 Free,pred HOV2 Pay,pred HOV3 Free,pred HOV3 Pay,pred Walk,pred BIKE,pred Walk to Transit,pred Park and Ride,pred Kiss and Ride,pred School Bus
pred Drive Alone Free,0.322314,0.000000,0.092715,0.027027,0.060811,0.000000,0.086420,0.085714,0.087912,0.075758,0.032258,0.000000
pred Drive Alone Pay,0.000000,0.391304,0.000000,0.459459,0.000000,0.183673,0.000000,0.000000,0.005495,0.007576,0.000000,0.000000
pred HOV2 Free,0.165289,0.000000,0.132450,0.054054,0.209459,0.000000,0.172840,0.042857,0.098901,0.015152,0.075269,0.059880
pred HOV2 Pay,0.000000,0.152174,0.013245,0.378378,0.000000,0.183673,0.018519,0.000000,0.005495,0.000000,0.005376,0.000000
pred HOV3 Free,0.057851,0.000000,0.145695,0.027027,0.351351,0.040816,0.123457,0.035714,0.093407,0.000000,0.080645,0.041916
pred HOV3 Pay,0.000000,0.108696,0.006623,0.216216,0.033784,0.591837,0.000000,0.000000,0.000000,0.007576,0.000000,0.000000
pred Walk,0.066116,0.000000,0.105960,0.027027,0.067568,0.000000,0.487654,0.157143,0.071429,0.022727,0.016129,0.041916
pred BIKE,0.090909,0.000000,0.000000,0.000000,0.006757,0.000000,0.098765,0.585714,0.093407,0.045455,0.021505,0.017964
pred Walk to Transit,0.123967,0.000000,0.026490,0.000000,0.020270,0.000000,0.117284,0.121429,0.302198,0.265152,0.139785,0.047904
pred Park and Ride,0.041322,0.000000,0.006623,0.054054,0.020270,0.020408,0.012346,0.050000,0.126374,0.462121,0.145161,0.000000


In [52]:
# accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.44838921761998685

In [53]:
# sensitivity
sensitivity = metrics.recall_score(y_test, y_pred, average = 'macro')
sensitivity

0.44642526036085844

In [54]:
# precision
precision = metrics.precision_score(y_test, y_pred, average = 'macro')
precision

0.4399668841365388

In [55]:
# F1-score
f1 = (2 * precision * sensitivity) / (precision + sensitivity)
f1

0.4431725439357991

In [56]:
# for the high income group
df_array = high_income_grp.values
X = df_array[:,0:34]
Y = df_array[:,34]

In [57]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [58]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], verbose = 0)

/home/likuwaha/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:49:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=64, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [59]:
y_pred = [round(value) for value in model.predict(X_test)]
y_test = [round(value) for value in y_test]

In [60]:
normalizeddf = pd.DataFrame(metrics.confusion_matrix(y_test, y_pred)/ metrics.confusion_matrix(y_test, y_pred).astype(np.float).sum(axis=1))
normalizeddf.columns = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf.index = ['pred Drive Alone Free','pred Drive Alone Pay','pred HOV2 Free',
                       'pred HOV2 Pay', 'pred HOV3 Free','pred HOV3 Pay','pred Walk','pred BIKE',
                       'pred Walk to Transit', 'pred Park and Ride','pred Kiss and Ride', 'pred School Bus']
normalizeddf

,pred Drive Alone Free,pred Drive Alone Pay,pred HOV2 Free,pred HOV2 Pay,pred HOV3 Free,pred HOV3 Pay,pred Walk,pred BIKE,pred Walk to Transit,pred Park and Ride,pred Kiss and Ride,pred School Bus
pred Drive Alone Free,0.434783,0.004292,0.105691,0.014019,0.057143,0.009132,0.08,0.032520,0.106383,0.078740,0.089744,0.011111
pred Drive Alone Pay,0.017391,0.639485,0.000000,0.247664,0.009524,0.114155,0.01,0.000000,0.000000,0.015748,0.000000,0.000000
pred HOV2 Free,0.208696,0.000000,0.235772,0.000000,0.304762,0.018265,0.12,0.040650,0.042553,0.015748,0.089744,0.044444
pred HOV2 Pay,0.026087,0.201717,0.016260,0.415888,0.019048,0.287671,0.02,0.000000,0.010638,0.015748,0.025641,0.011111
pred HOV3 Free,0.078261,0.004292,0.113821,0.004673,0.438095,0.027397,0.09,0.040650,0.053191,0.000000,0.064103,0.044444
pred HOV3 Pay,0.017391,0.098712,0.016260,0.341121,0.028571,0.515982,0.01,0.000000,0.000000,0.007874,0.012821,0.000000
pred Walk,0.113043,0.000000,0.065041,0.000000,0.038095,0.004566,0.39,0.113821,0.170213,0.000000,0.012821,0.044444
pred BIKE,0.086957,0.004292,0.048780,0.000000,0.047619,0.000000,0.18,0.536585,0.010638,0.055118,0.051282,0.055556
pred Walk to Transit,0.017391,0.000000,0.040650,0.004673,0.047619,0.000000,0.05,0.048780,0.563830,0.055118,0.115385,0.011111
pred Park and Ride,0.147826,0.021459,0.016260,0.000000,0.019048,0.004566,0.03,0.032520,0.031915,0.582677,0.205128,0.000000


In [61]:
# accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.5021591610117212

In [62]:
# sensitivity
sensitivity = metrics.recall_score(y_test, y_pred, average = 'macro')
sensitivity

0.49794327577888337

In [63]:
# precision
precision = metrics.precision_score(y_test, y_pred, average = 'macro')
precision

0.4903142866011782

In [64]:
# F1-score
f1 = (2 * precision * sensitivity) / (precision + sensitivity)
f1

0.4940993346782665

Comparing the predictions on different income groups, we see that our predictions are slightly better for the higher income groups. For the higher income group, accuracy is higher, and so is sensitivity and precision. It is more noticeable for accuracy. 

For both, sensitivity is higher than precision, which means there is better predictions on the positive class that are correctly predicted, being sligtly better than rate as which precision is correct. 

The predictions individually, by travel modes, we see higher accuracy in predicting Drive Alone Pay and predictions for School Bus. Unfortunately, our predictions for the low income group is not great compared to higher income groups, and so our predictions in general may not be as reliable in representing that group. In terms of how much data we have for each group, the low income group and high income group dataframes have similar amount of rows, above 4,500 rows and under 5,000. Thus, the accuracy has nothing to do with having the data under-represented. 